In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']


### Data Load

In [2]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')
seed = 1208
np.random.seed(seed)

In [11]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_hist = utils.read_df_pkl('../input/hist*0*')
# df_new = utils.read_df_pkl('../input/new*0*')

100%|██████████| 3/3 [00:00<00:00, 165.33it/s]


SystemExit: 

In [25]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [4]:
# for df in [df_hist,df_new]:
for df in [df_hist]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

In [29]:
auth1_hist = df_hist[df_hist.authorized_flag==1]
auth0_hist = df_hist[df_hist.authorized_flag==0]
auth1_new = df_new[df_new.authorized_flag==1]
auth0_new = df_new[df_new.authorized_flag==0]

In [90]:
new_aggs = {}
new_aggs['purchase_date'] = ['max','min']

new_columns = get_new_columns('hist', new_aggs)
tmp = df_hist[['card_id', 'purchase_date']].groupby('card_id').agg(new_aggs)
tmp.columns = new_columns

tmp['hist_term'] = (tmp['hist_purchase_date_max'] - tmp['hist_purchase_date_min']).dt.days

SystemExit: 

### 117 auth1_hist feature

In [120]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

df = auth1_hist
fname = 'auth1_hist'
aggs = {}

aggs['month'] = ['nunique']
aggs['dayofweek'] = ['nunique']
aggs['purchase_amount'] = ['sum','max','mean']
aggs['purchase_date'] = ['max','min']
aggs['month_diff'] = ['mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns(fname, aggs)
df_agg = df.groupby('card_id').agg(aggs)
df_agg.columns = new_columns
df_agg.reset_index(drop=False,inplace=True)

df_agg['auth1_hist_term'] = (df_agg['auth1_hist_purchase_date_max'] - df_agg['auth1_hist_purchase_date_min']).dt.days
df_agg = df_agg.merge(tmp, how='inner', on=key)
df_agg[f'{fname}_term_ratio'] = df_agg['auth1_hist_term'] / df_agg['hist_term']

df_agg['auth1_hist_term_freq'] = df_agg['auth1_hist_card_id_size']/df_agg['auth1_hist_term']
df_train = df_train.merge(df_agg,on='card_id',how='left')
df_test = df_test.merge(df_agg,on='card_id',how='left')
del df_agg
gc.collect()
df_train.head()


100%|██████████| 3/3 [00:00<00:00, 173.89it/s]

,first_active_month,card_id,feature_1,feature_2,feature_3,target,auth1_hist_month_nunique,auth1_hist_dayofweek_nunique,auth1_hist_purchase_amount_sum,auth1_hist_purchase_amount_max,auth1_hist_purchase_amount_mean,auth1_hist_purchase_date_max,auth1_hist_purchase_date_min,auth1_hist_month_diff_mean,auth1_hist_category_1_sum,auth1_hist_category_1_mean,auth1_hist_card_id_size,auth1_hist_category_2_mean_mean,auth1_hist_category_3_mean_mean,auth1_hist_term,hist_purchase_date_max,hist_purchase_date_min,hist_term,auth1_hist_term_ratio,auth1_hist_term_freq
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,9,7,-157.397016,2.258395,-0.637235,2018-02-25 09:31:15,2017-06-27 14:18:08,10.165992,0,0.000000,247,-0.605846,-0.641168,242,2018-02-25 09:31:15,2017-06-27 14:18:08,242,1.0,1.020661
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,12,7,-208.883450,4.630299,-0.616175,2018-01-31 22:31:09,2017-01-06 16:29:42,11.165192,29,0.085546,339,-0.606068,-0.495468,390,2018-01-31 22:31:09,2017-01-06 16:29:42,390,1.0,0.869231
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,10,7,-27.828424,-0.145847,-0.678742,2018-02-27 19:08:25,2017-01-11 08:21:22,10.097561,0,0.000000,41,-0.589559,-0.641168,412,2018-02-27 19:08:25,2017-01-11 08:21:22,412,1.0,0.099515
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,6,7,-49.491364,1.445596,-0.642745,2018-02-28 11:44:40,2017-09-26 16:22:21,10.116883,12,0.155844,77,-0.594682,-0.554738,154,2018-02-28 11:44:40,2017-09-26 16:22:21,154,1.0,0.500000
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,4,7,-69.040465,6.992617,-0.539379,2018-02-28 20:40:41,2017-11-12 00:00:00,10.101562,12,0.093750,128,-0.593270,-0.591703,108,2018-02-28 20:40:41,2017-11-12 00:00:00,108,1.0,1.185185


In [121]:
feat_no = '117_ah1_'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if (col.count('feature_')):continue
    if (col.count('auth0_')):continue
    if not(col.count('term_ratio')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)

### 118 auth0_hist feature

In [122]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

df = auth0_hist
fname = 'auth0_hist'
aggs = {}

aggs['purchase_amount'] = ['sum','max','mean']
aggs['purchase_date'] = ['max','min']
aggs['month_diff'] = ['mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns(fname, aggs)
df_agg = df.groupby('card_id').agg(aggs)
df_agg.columns = new_columns
df_agg.reset_index(drop=False,inplace=True)

df_agg[f'{fname}_term'] = (df_agg[f'{fname}_purchase_date_max'] - df_agg[f'{fname}_purchase_date_min']).dt.days
df_agg = df_agg.merge(tmp, how='inner', on=key)
df_agg[f'{fname}_term_ratio'] = df_agg['auth0_hist_term'] / df_agg['hist_term']

df_agg[f'{fname}_term_freq'] = df_agg[f'{fname}_card_id_size']/df_agg[f'{fname}_term']
df_train = df_train.merge(df_agg,on='card_id',how='left')
df_test = df_test.merge(df_agg,on='card_id',how='left')
del df_agg
gc.collect()
df_train.head()


100%|██████████| 3/3 [00:00<00:00, 166.12it/s]

,first_active_month,card_id,feature_1,feature_2,feature_3,target,auth0_hist_purchase_amount_sum,auth0_hist_purchase_amount_max,auth0_hist_purchase_amount_mean,auth0_hist_purchase_date_max,auth0_hist_purchase_date_min,auth0_hist_month_diff_mean,auth0_hist_category_1_sum,auth0_hist_category_1_mean,auth0_hist_card_id_size,auth0_hist_category_2_mean_mean,auth0_hist_category_3_mean_mean,auth0_hist_term,hist_purchase_date_max,hist_purchase_date_min,hist_term,auth0_hist_term_ratio,auth0_hist_term_freq
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,-8.571723,-0.431922,-0.659363,2017-12-27 14:33:33,2017-07-15 14:58:10,10.153846,0.0,0.000000,13.0,9.817255,9.698610,164.0,2018-02-25 09:31:15,2017-06-27 14:18:08,242.0,0.677686,0.079268
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,-1.122886,1.942838,-0.102081,2017-12-28 13:21:18,2017-03-03 21:27:17,11.272727,2.0,0.181818,11.0,9.817255,1.043947,299.0,2018-01-31 22:31:09,2017-01-06 16:29:42,390.0,0.766667,0.036789
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,-1.338967,-0.637515,-0.669484,2017-04-21 19:00:14,2017-02-23 19:32:09,10.000000,0.0,0.000000,2.0,6.764233,13.296816,56.0,2018-02-27 19:08:25,2017-01-11 08:21:22,412.0,0.135922,0.035714
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,20.352809,7.193041,4.070562,2018-02-27 19:21:23,2018-01-20 21:49:13,10.000000,3.0,0.600000,5.0,5.393454,1.075873,37.0,2018-02-28 20:40:41,2017-11-12 00:00:00,108.0,0.342593,0.135135


SystemExit: 

In [123]:
feat_no = '118_ah0_'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if (col.count('feature_')):continue
    if (col.count('purchase_date')):continue
    if (col.count('auth1_hist_')):continue
    if not(col.count('term_ratio')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)

### 119 auth1 auth0 date max, min diff

In [64]:
feat_no = '119_ath_'
df_train['auth1_0_hist_purchase_date_max_diff'] = (df_train['auth1_hist_purchase_date_max'] -  df_train['auth0_hist_purchase_date_max']).map(lambda x: x.days)
df_train['auth1_0_hist_purchase_date_min_diff'] = (df_train['auth1_hist_purchase_date_min'] -  df_train['auth0_hist_purchase_date_min']).map(lambda x: x.days)
df_test['auth1_0_hist_purchase_date_max_diff'] = (df_test['auth1_hist_purchase_date_max'] -  df_test['auth0_hist_purchase_date_max']).map(lambda x: x.days)
df_test['auth1_0_hist_purchase_date_min_diff'] = (df_test['auth1_hist_purchase_date_min'] -  df_test['auth0_hist_purchase_date_min']).map(lambda x: x.days)

ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if not(col.count('auth1_0_hist')):continue
#     if not(col.count('card_id_total')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)

In [31]:
new_aggs = {}
new_aggs['authorized_flag'] = ['sum', 'mean']

SystemExit: 

### 120 初期3ヶ月のauthorized_meanと直近3ヶ月のauthorized_mean, そのdiff

In [5]:
df_hist['yyyymm'] = df_hist['purchase_date'].map(lambda x: str(x)[:7])
df_hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,year,month,dayofweek,hour,month_diff,yyyymm
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,2017,6,6,15,10,2017-06
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,2017,7,5,12,10,2017-07
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,2017,8,2,22,10,2017-08
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,2017,9,5,10,11,2017-09
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,2017,3,4,1,11,2017-03


In [6]:
# 初期3ヶ月
df_auth = df_hist[[key, 'yyyymm', 'authorized_flag']]
df_auth.sort_values(by=[key, 'yyyymm'], inplace=True)
df_auth_row = utils.row_number(df=df_auth[[key, 'yyyymm']].drop_duplicates(), level=[key])
df_auth_row = df_auth_row.merge(df_hist[[key, 'yyyymm', 'authorized_flag']], how='inner', on=[key, 'yyyymm'])

df_first3 = df_auth_row[df_auth_row['row_no']<=3]

df_first3.head(10)

,card_id,yyyymm,row_no,authorized_flag
0,C_ID_00007093c1,2017-02,1,1
1,C_ID_00007093c1,2017-02,1,1
2,C_ID_00007093c1,2017-02,1,1
3,C_ID_00007093c1,2017-02,1,1
4,C_ID_00007093c1,2017-03,2,1
5,C_ID_00007093c1,2017-03,2,1
6,C_ID_00007093c1,2017-03,2,1
7,C_ID_00007093c1,2017-03,2,1
8,C_ID_00007093c1,2017-03,2,0
9,C_ID_00007093c1,2017-03,2,0


In [7]:
# 後期3ヶ月
# df_auth = df_hist[[key, 'yyyymm', 'authorized_flag']]
df_auth.sort_values(by=[key, 'yyyymm'], ascending=False, inplace=True)
df_auth_row = utils.row_number(df=df_auth[[key, 'yyyymm']].drop_duplicates(), level=[key])
df_auth_row = df_auth_row.merge(df_hist[[key, 'yyyymm', 'authorized_flag']], how='inner', on=[key, 'yyyymm'])

df_last3 = df_auth_row[df_auth_row['row_no']<=3]

df_last3.head(10)

,card_id,yyyymm,row_no,authorized_flag
0,C_ID_fffffd5772,2018-02,1,1
1,C_ID_fffffd5772,2018-02,1,1
2,C_ID_fffffd5772,2018-02,1,1
3,C_ID_fffffd5772,2018-02,1,1
4,C_ID_fffffd5772,2018-02,1,0
5,C_ID_fffffd5772,2018-02,1,0
6,C_ID_fffffd5772,2018-02,1,0
7,C_ID_fffffd5772,2018-02,1,1
8,C_ID_fffffd5772,2018-02,1,0
9,C_ID_fffffd5772,2018-02,1,1


In [12]:
# df_first3_auth = df_first3.groupby(key)['authorized_flag'].mean()
# df_first3_auth.name = 'first_3month_authorized_flag_mean'

# df_last3_auth = df_last3.groupby(key)['authorized_flag'].mean()
# df_last3_auth.name = 'last_3month_authorized_flag_mean'

df_train = df_train.merge(df_first3_auth.reset_index(), how='left', on=key)
df_train = df_train.merge(df_last3_auth.reset_index(), how='left', on=key)
df_test = df_test.merge(df_first3_auth.reset_index(), how='left', on=key)
df_test = df_test.merge(df_last3_auth.reset_index(), how='left', on=key)

df_train['diff_last_first_authorized_flag_mean'] = df_train['last_3month_authorized_flag_mean'] -  df_train['first_3month_authorized_flag_mean']
df_test['diff_last_first_authorized_flag_mean'] = df_test['last_3month_authorized_flag_mean'] -  df_test['first_3month_authorized_flag_mean']

df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_3month_authorized_flag_mean,last_3month_authorized_flag_mean,diff_last_first_authorized_flag_mean
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,0.947917,0.980198,0.032281
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,0.982143,0.982456,0.000313
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,0.941176,1.000000,0.058824
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,1.000000,1.000000,0.000000
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,0.979592,0.955357,-0.024235


In [14]:
feat_no = '120_ath_'

ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if not(col.count('flag_mean')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)